In [12]:
import sbfit 
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt



# sb profile and Fitting

In [3]:
def DoubleBeta(x, beta, rc1, rc2, ratio, norm, const):
    out = norm * (np.power(1 + (x / rc1) **2, -3 * beta + 0.5) + ratio * np.power(1 + (x / rc2) **2, -3 * beta + 0.5)) + const
    return out
def BetaModel(x, beta, rc, norm,const):
    out = norm * np.power(1. + (x / rc) ** 2, -3. * beta + 0.5) + const
    return out

In [14]:
datapath = '/Users/eusracenorth/Documents/suzaku_A222-223/profile_230302'
for ins in [ 'xmm']:#'chandra',
    for band in ['0.5-7', '0.7-1.2']:
        for clus in ['A222','A223']:
            print(f'{ins}, {band}, {clus}')
            path = f'{datapath}/{ins}'
            if ins=='chandra':
                reg_list = sbfit.read_region(f"{path}/{clus}_{ins}.reg")
                obs = sbfit.Observation(f'{path}/images/{band}_band1_thresh.img',f'{path}/../{band}_band1_thresh_{clus}filamsk_chan.expmap',f'{path}/images/{band}_band1_nxb_scaled.img')
            elif ins=='xmm':
                reg_list = sbfit.read_region(f"{path}/{clus}_{ins}.reg")
                if band =='0.5-7':
                    obs = sbfit.Observation(f'{path}/{band}/EPIC_cts.img',f'{path}/../EPIC_exp_{clus}filamsk_xmm.expmap',f'{path}/{band}/EPIC_bkg.img')
                elif band =='0.7-1.2':
                    obs = sbfit.Observation(f'{path}/{band}/EPIC_cts_700-1200.img',f'{path}/../EPIC_exp_700-1200_{clus}filamsk_xmm.expmap',f'{path}/{band}/EPIC_bkg_700-1200.img')
                
            obs_list = sbfit.ObservationList([obs])
            pro = obs_list.get_profile(reg_list)
            pro.rebin(1, 700, method="min_cts", min_cts=70)
            pro.plot(plot_type="Profile")
            beta = sbfit.model.custom_model(DoubleBeta)

            pro.set_model(beta)
            # set parameters
            pro.model.norm = 1e-4
            pro.model.beta = 0.6
            pro.model.rc1 = 60
            pro.model.rc2 = 10
            pro.model.ratio = 1
            pro.model.norm = 1e-4
            pro.model.const = np.power(10.,-6.71)
            # calculate model parameter
            pro.calculate()
            # plot the model profile together with the observed profile
            pro.plot()
            pro.model.norm.bounds = (5e-7, 1e-3)
            pro.model.beta.bounds = (0.2, 0.9)
            pro.model.rc1.bounds = (1, 200)
            pro.model.rc2.bounds = (1, 200)
            pro.model.ratio = (1e-5, 100)
            pro.model.const.bounds = (1e-10, 1e-2)
            pro.fit(show_step=False, show_result=True, tolerance=0.01)
            pro.plot()
            pro.binned_profile.write(f"{datapath}/{ins}_{band}_{clus}_sbfit_profile.fits", overwrite=True)


xmm, 0.5-7, A222


KeyError: "Keyword 'EXPOSURE' not found."

In [ ]:
# plot sbfit data

def BetaModel(x, beta, rc, norm):
    out = norm * np.power(1. + (x / rc) ** 2, -3. * beta + 0.5) 
    return out

data = Table.read("a223_sbfit_profile.fits")
fig, (ax1,ax2) = plt.subplots(2,1,figsize = (8,8), gridspec_kw={'height_ratios': [ 3, 1]})
ax1.errorbar(data['r']/60, data['sb'], yerr = data['sb_error'], xerr = data['r_error_left']/60, label = 'data')
ax1.errorbar(data['r']/60, data['bkg_sb'], yerr = data['bkg_sb_error'], xerr = data['r_error_left']/60, label = 'nxb')
ax1.axvline(7, label = 'r200')
ax1.axvline(5, label = 'r500')
norm = 7.11e-05
beta = 4.87e-01
rc = 3.16e+01/60
bkg = 1.00e-10
y1 = BetaModel(data['r']/60, beta, rc, norm)
ax1.set_title('sbfit a222, mincts 70, b/a = 0.73')
ax1.plot(data['r']/60, y1, 'r-', label = 'sbfit')
ax1.set_xlabel('r [arcmin]')
ax1.set_ylabel('SB [$cts/s/arcmin^2$]')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax2.errorbar(data['r']/60, data['sb']-y1, yerr = (data['sb_error']**2 + data['bkg_sb_error']**2)**0.5, xerr = data['r_error_left']/60, fmt = '.')
ax2.set_xscale('log')
ax2.axhline(0, linestyle = '--', color = 'orange')
# ax2.set_yscale('log')
ax1.legend()
plt.show()
plt.close()
